In [6]:
import tokenizers
import datasets
import tqdm
import torch
import os

vocab_size = 2**13
vocab_name = '8k'
output_dir = f'tiny-stories-{vocab_name}-eos'
os.makedirs(output_dir, exist_ok=True)

In [7]:
ds = datasets.load_dataset("roneneldan/TinyStories")

In [8]:
s = 0
for t in ds['validation']['text'][:3]:
    s += len(t)
    print(t)
    
print(s)

Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."

After playing with the car, Kitty and Spot felt thirsty. They found a small pond with clear water. They drank the water and felt very happy. They played together all day and became best friends.
Once upon a time, in a big forest, there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, rocks, and hills. One day, Roxy found an icy hill. She had never seen anything like it before. It was shiny and cold, and she wanted to climb it.

Roxy tried to climb the icy hill, but it was very slippery. She tried again and again, but she kept falling down. Roxy was sad. She wanted to climb the icy hill so much. Then, she saw a little bird named Billy. Billy saw that Roxy was sad and asked, "Why are you sad, Roxy?"

Roxy told Billy about the icy hill and how she couldn't climb it. Billy said, "I have an idea! Let's find some big 

In [9]:
t = tokenizers.ByteLevelBPETokenizer()

def eos_iterator(ds):
    for ex in ds:
        yield ex + '[EOS]'

t.train_from_iterator(eos_iterator(ds['train']['text']), vocab_size=vocab_size, min_frequency=2, special_tokens=['[EOS]'])
t.save_model(output_dir, 'tokenizer')

['tiny-stories-8k-eos/tokenizer-vocab.json',
 'tiny-stories-8k-eos/tokenizer-merges.txt']

In [10]:
for split in ['train', 'validation']:
    output_tokens = []
    for story in tqdm.tqdm(eos_iterator(ds[split]['text']), desc=split):    
        output_tokens.extend(t.encode(story).ids)
    if split == 'train':
        print(f'{vocab_size=} makes for {len(output_tokens)}')
    torch.save(torch.tensor(output_tokens, dtype=torch.int16), f'{output_dir}/{split}.pt')

train: 2119719it [15:13, 2319.43it/s]


vocab_size=8192 makes for 466747279


validation: 21990it [00:07, 2901.71it/s]


# Train data size (chars) 1899973203

# 4k vocab ->  483664338 tokens -> 3.928 chars per token .  approx 1030 Mtok/sec with a 10.8 M model
# 8k vocab ->  464614341 tokens -> 4.089 chars per token .  approx 950 Mtok/sec with a 11.6M model 
# 16k vocab -> 460959001